In [9]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [10]:
#data = pd.read_csv('ausgrid/customer_1.csv', index_col=0, sep=";")
data = pd.read_csv('../data/load/dataid171.csv', index_col=0, sep=";")
data.loc[data['loadProd'] < 0, 'loadProd'] = 0
data.head()

,date,time,dateTime,dayOfWeek,holiday,dayId,loadCons,loadProd,loadGrid
id,,,,,,,,,
1,2014-01-01,00:00:00,2014-01-01 00:00:00,3,1,1,0.885267,0.0,0.885267
2,2014-01-01,01:00:00,2014-01-01 01:00:00,3,1,1,0.512767,0.0,0.512767
3,2014-01-01,02:00:00,2014-01-01 02:00:00,3,1,1,0.448867,0.0,0.448867
4,2014-01-01,03:00:00,2014-01-01 03:00:00,3,1,1,0.473483,0.0,0.473483
5,2014-01-01,04:00:00,2014-01-01 04:00:00,3,1,1,0.440150,0.0,0.440150


In [11]:
gen = pd.DataFrame(data[['loadProd']])
con = pd.DataFrame(data[['loadCons']])
con = con['loadCons']
gen = gen['loadProd']

In [12]:
#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)
def funcG(xg):
    dim = len(x)
    bat_actions = []
    grid_actions = []
    bat_actions.append(x[0]-bat_t0)
    grid_actions.append(demand[0]-pv_gen[0]+bat_actions[0])
    bat_action = x[0]-bat_t0
    grid_sum=demand[0]-pv_gen[0]+bat_action
    for i in range(1,dim):
        bat_actions.append(x[i]-x[i-1]) 
        grid_actions.append(demand[i]-pv_gen[i]+bat_actions[i])
        bat_action = x[i]-x[i-1]
        grid_sum+=demand[i]-pv_gen[i]+bat_action
    print("pos ",x)
    print("bat_actions ",bat_actions)
    print("demand ",demand)
    print("pv_gen ",pv_gen)
    print("grid_actions ",grid_actions)
    print("grid_sum ",grid_sum)
    print("bat_t0 ",bat_t0)
    print()
    return xg[0]
        
def cost_function(x,demand,pv_gen,bat_t0,bat_act0):
    dim = len(x)
    bat_action = x[0]-bat_t0
    grid_sum=demand[0]-pv_gen[0]+bat_action
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        grid_sum+=demand[i]-pv_gen[i]+bat_action
    return grid_sum

def cost_function2(x,demand,pv_gen,bat_t0,bat_act0):
    dim = len(x)
    bat_action = x[0]-bat_t0
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
    return grid_sum,0

def cost_function22(x,demand,pv_gen,bat_t0,bat_act0):
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
    return grid_sum,0

def charging_interrupt(bat_action, bat_act0):
    if bat_act0==1:
        if bat_action>=0:
            return 0
        else:
            return 1
    if bat_act0==-1:
        if bat_action<=0:
            return 0
        else:
            return 1
        
def cost_function4(x,demand,pv_gen,bat_t0,bat_act0):
    batActions = []
    score = 0
    dim = len(x)
    bat_action = x[0]-bat_t0
    batActions.append(bat_action)
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        batActions.append(bat_action)
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
        
    if bat_act0==1:
        if batActions[0]>0:
            score+=batActions[0]
        elif batActions[0]<0:
            if bat_t0<13:
                score-=1
            score+=abs(batActions[0])
            bat_act0=-1
    elif bat_act0==-1:
        if batActions[0]<0:
            score+=abs(batActions[0])
        elif batActions[0]>0:
            if bat_t0>0:
                score-=1
            score+=batActions[0]
            bat_act0=1
            
    for i in range(1,dim):
        if bat_act0==1:
            if batActions[i]>0:
                score+=batActions[i]
            elif batActions[i]<0:
                if x[i-1]<13:
                    score-=1
                score+=abs(batActions[i])
                bat_act0=-1
        elif bat_act0==-1:
            if batActions[i]<0:
                score+=abs(batActions[i])
            elif batActions[i]>0:
                if x[i-1]>0:
                    score-=1
                score+=batActions[i]
                bat_act0=1
    grid_sum-=(score*0.8)
    return (grid_sum),bat_act0

def cost_function3(x,demand,pv_gen,bat_t0,bat_act0):
    dim = len(x)
    bat_action = x[0]-bat_t0
    interrupt = charging_interrupt(bat_action, bat_act0)
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)+interrupt
    if interrupt == 1:
        if bat_act0 == 1:
            bat_act0 = -1
        else:
            if bat_act0 == -1:
                bat_act0 = 1
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        interrupt = charging_interrupt(bat_action, bat_act0)
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)+interrupt/4
        if interrupt == 1:
            if bat_act0 == 1:
                bat_act0 = -1
        else:
            if bat_act0 == -1:
                bat_act0 = 1
    return grid_sum, bat_act0

In [13]:
class Battery:
    def __init__(self,cap,soc,act):
        self.capacity=cap
        self.SOC=soc
        self.charg=act

In [14]:
class Particle:
    def __init__(self,x0,demand,pv_gen,bat_t0,bat_act0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        self.dem=demand
        self.pv_gen=pv_gen
        self.bat_t0=bat_t0          #som pridal
        self.bat_act0=bat_act0      #som pridal
        self.dist_best_i=-1         #som pridal
        self.bat_best_act=bat_act0
        self.sthr=0.2

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])
            #self.pos_best_i.append(x0[i])#som pridal

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i,self.bat_act0 = costFunc(self.position_i,self.dem,self.pv_gen,self.bat_t0,self.bat_act0)

        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
            self.bat_best_act = self.bat_act0

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()
            #print("ahoj",self.pos_best_i[0])
            #print("posbestg",pos_best_g[i])
            #print("i",i)
            #print("selfposbest",self.pos_best_i[i])
            #print("selfpositioni",self.position_i[i])
            #print("diff",self.pos_best_i[i]-self.position_i[i])
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

In [15]:
#PSO(cost_function,initial,BSSBounds,num_particles=1000,maxiter=50,demand=conWindow,pv_gen=genWindow,BSSTime0)

class PSO:
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter,demand,pv_gen,bat_t0,bat_act0):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        self.pos_best_g=[]                   # best position for group
        #pos_best_g.append(x0[0])        #som pridal
        #pos_best_g.append(x0[0])        #som pridal
        dist_best_g=-1                  #som pridal
        thr = 0.2
        dim = len(self.pos_best_g)
        bat_actions = []
        self.grid_actions = []
        self.pv_actions = []
        self.demand_sum = 0
        self.pvToDemand=0
        self.batToDemand=0
        self.gridToDemand=0
        self.grid_total = 0
        self.grid_action = 0
        self.bat_final_act = 1
        # establish the swarm
        swarm=[]
        #print(demand)
        #print(pv_gen)
        #print(bat_t0)
        for i in range(0,num_particles):
            swarm.append(Particle(x0,demand,pv_gen,bat_t0,bat_act0))

        # begin optimization loop
        i=0
        while i < maxiter:
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                #print("particle ",j)
                swarm[j].evaluate(costFunc)

                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    #print('ahoj',swarm[j].position_i)
                    self.pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)
                    self.bat_final_act = swarm[j].bat_best_act

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(self.pos_best_g)
                swarm[j].update_position(bounds)
            #print("sevas")
            i+=1
            
        dim = len(self.pos_best_g)
        bat_actions = []
        self.grid_actions = []
        pv_actions = []
        bat_actions.append(self.pos_best_g[0]-bat_t0)
        self.grid_actions.append(demand[0]-pv_gen[0]+bat_actions[0])
        #pv_actions.append()
        self.demand_sum = demand[0]
        self.grid_total = self.grid_actions[0]
        for i in range(1,dim):
            bat_actions.append(self.pos_best_g[i]-self.pos_best_g[i-1])
            self.grid_actions.append(demand[i]-pv_gen[i]+bat_actions[i])
            self.demand_sum+=demand[i]
            self.grid_total+=self.grid_actions[i]
            
        for i in range(0,dim):
            if pv_gen[i] >= demand[i]:
                self.pvToDemand += demand[i]
                self.pv_actions.append(demand[i])
            else:
                self.pvToDemand += pv_gen[i]
                self.pv_actions.append(pv_gen[i])
                
            zeroStateRest = demand[i]-pv_gen[i]
            if bat_actions[i]<0 and zeroStateRest>0:
                if abs(bat_actions[i])>=zeroStateRest:
                    self.batToDemand+=zeroStateRest
                else:
                    self.batToDemand+=abs(bat_actions[i])
                    
            if self.grid_actions[i]>0 and pv_gen[i]<demand[i]:
                conRest = demand[i]-pv_gen[i]+bat_actions[i]
                #self.gridToDemand+=conRest
                if bat_actions[i] >= 0:
                    self.gridToDemand+=demand[i]-pv_gen[i]
                else:
                    if abs(bat_actions[i])+pv_gen[i] < demand[i]:
                        self.gridToDemand+=conRest
            
            
            
        # print final results
        #print("demand ",demand)
        #print("pv generation ",pv_gen)
        #print("bat_actions ",bat_actions)
        #print("best pos ",self.pos_best_g)
        
        
        #print("grid_actions ",self.grid_actions) 
        #print("best cost ",err_best_g)
        #print()
        #print("grid total ",self.grid_total)
        #print("Total demand ",self.demand_sum)
        #print("battery SOC ",bat.SOC)
        bat.charg = self.bat_final_act
        bat.SOC=self.pos_best_g[dim-1]
        self.grid_action = self.pos_best_g[0]
        #print("battery SOC after ",bat.SOC)
        #print()
        #total_d+=demand_sum
        #total_g+=grid_totalabs

In [8]:
if __name__ == "__PSO__":
    main()

In [16]:
#--- RUN ----------------------------------------------------------------------+
windowSize=4
start=1
#end=48*7
end = 8761
bat=Battery(13, 0, 1)
total_demand = 0
total_grid = 0
pv_to_demand = 0
bat_to_demand = 0
grid_to_demand = 0

gridTotalProgress = []
pvTotalProgress = []
batSOCProgress = []
batSOCProgress.append(0)

for c in range(start,end,windowSize):
    d=c+windowSize
    initial= []
    genWindow = []
    conWindow = []
    BSSBounds = []
    for i in range(c,d):
        if i<=end:
            genWindow.append(gen[i])
            conWindow.append(con[i])
            initial.append(0)#bat.capacity/2)+
            BSSBounds.append((0,bat.capacity))
            if i % 50 == 0:
                print(i)
    psoOpt = PSO(cost_function22,initial,BSSBounds,num_particles=1000,maxiter=100,demand=conWindow,pv_gen=genWindow,bat_t0=bat.SOC, bat_act0=bat.charg)
    l = len(psoOpt.grid_actions)
    for j in range(0,l):
        gridTotalProgress.append(psoOpt.grid_actions[j])
        pvTotalProgress.append(psoOpt.pv_actions[j])
        batSOCProgress.append(psoOpt.pos_best_g[j])
    total_demand+=psoOpt.demand_sum
    total_grid+=psoOpt.grid_total
    pv_to_demand+=psoOpt.pvToDemand
    bat_to_demand+=psoOpt.batToDemand
    grid_to_demand+=psoOpt.gridToDemand
print('FINAL:')
print('Total demand: ',total_demand)
print('Total grid: ',total_grid)
print('pv to demand: ',pv_to_demand)
print('battery to demand: ',bat_to_demand)
print('grid to demand: ',grid_to_demand)

print('demand meet: ',pv_to_demand+bat_to_demand+grid_to_demand)

#print('grid actions: ',gridTotalProgress)
#print('battery socs: ',batSOCProgress)

#toto tu bolo pred bat1
#data = pd.read_csv('data/load/dataid187.csv', index_col=0, sep=";")
#data.loc[data['loadProd'] < 0, 'loadProd'] = 0
#gen = pd.DataFrame(data[['loadProd']])
#con = pd.DataFrame(data[['loadCons']])
#con = con['loadCons']
#gen = gen['loadProd']






g=0
gi=0
ge=0

b=0
bc=0
bd=0

for i in(gridTotalProgress):
    if i >= 0:
        gi=gi+i
    if i < 0:
        ge=ge-i
    g=g+i

l = len(batSOCProgress)
print('len ', l)
for i in range(1,l):
    action = batSOCProgress[i]-batSOCProgress[i-1]
    if action > 0:
        bc=bc+action
    if action <= 0:
        bd=bd-action
    b=b+action
        
print(g)
print(gi)
print(ge)

print()

#print(batSOCProgress)
print(b)
print(bc)
print(bd)#111.2 85.49

np.savetxt("psoVymenaGridRokDom171.csv", gridTotalProgress, delimiter=",")
np.savetxt("psoVymenaBatRokDom171.csv", batSOCProgress, delimiter=",")

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
FINAL:
Total demand:  7771.249158333324
Total grid:  100.03675833333526
pv to demand:  3518.0897833333306
battery to demand:  1796.781480647485
g

In [63]:
for i in range(1,5,2):
    print(i)

1
3


In [15]:
#--- RUN ----------------------------------------------------------------------+
#windowSize=4
#start=1
##end=48*7
#end = 8761
#bat=Battery(13, 0, 1)
#total_demand = 0
#total_grid = 0

#gridTotalProgress2 = []
#batSOCProgress2 = []
#batSOCProgress2.append(0)

#for c in range(start,end,windowSize):
    #d=c+windowSize
    #initial= []
    #genWindow = []
    #conWindow = []
    #BSSBounds = []
    #for i in range(c,d):
     #   if i<=end:
      #      genWindow.append(gen[i])
       #     conWindow.append(con[i])
        #    initial.append(0)#bat.capacity/2)
         #   BSSBounds.append((0,bat.capacity))
          #  if i % 50 == 0:
          #      print(i)
    #psoOpt = PSO(cost_function2,initial,BSSBounds,num_particles=1000,maxiter=100,demand=conWindow,pv_gen=genWindow,bat_t0=bat.SOC)
    #l = len(psoOpt.grid_actions)
    #for j in range(0,l):
    #    gridTotalProgress2.append(psoOpt.grid_actions[j-1])
    #    batSOCProgress2.append(psoOpt.pos_best_g[j-1])
    #total_demand+=psoOpt.demand_sum
    #total_grid+=psoOpt.grid_total
#print('FINAL:')
#print('Total demand: ',total_demand)
#print('Total grid: ',total_grid)

In [54]:
sum(pvTotalProgress)

16.450550000000003

vypocet

In [76]:
g=0
gi=0
ge=0

b=0
bc=0
bd=0

for i in(gridTotalProgress):
    if i >= 0:
        gi=gi+i
    if i < 0:
        ge=ge-i
    g=g+i

l = len(batSOCProgress)
print('len ', l)
for i in range(1,l):
    action = batSOCProgress[i]-batSOCProgress[i-1]
    if action > 0:
        bc=bc+action
    if action <= 0:
        bd=bd-action
    b=b+action
        
print(g)
print(gi)
print(ge)

print()

#print(batSOCProgress)
print(b)
print(bc)
print(bd)#111.2 85.49

np.savetxt("psoVymenaGridRokDom93.csv", gridTotalProgress, delimiter=",")
np.savetxt("psoVymenaBatRokDom93.csv", batSOCProgress, delimiter=",")

len  8761
5937.842816379295
7119.518413843578
1181.675597464288

-8.881784197001252e-15
2138.7244746114093
2138.7244746114097


skusanie

In [295]:
#gen.describe()
pv=0.669
bat.SOC=0.2
boundsGrid=(0,3)
boundsPV=(0,pv)
boundsBSS=(-bat.SOC,bat.capacity-bat.SOC)

func2([0.679, 0.669, 0], 1.348,[boundsGrid,boundsPV,boundsBSS])
func2([0.479, 0.669, -0.2], 1.348,[boundsGrid,boundsPV,boundsBSS])
#abs(xbss[2]-(bounds[1][1]-xbss[1]))
#(abs(abs(xbss[2])+xbss[1]-fdembss))



NameError: name 'func2' is not defined

In [ ]:
func2([0.479, 0.669, -0.2], 1.348,[boundsGrid,boundsPV,boundsBSS])

In [34]:
for i in range(14,17):
    print(gen[i])

0.0
0.006
0.019


In [94]:
gridTotalProgress[8759]

0.6383691362524451

In [77]:
np.savetxt("psoVymenaGridRokDom93.csv", gridTotalProgress, delimiter=",")
np.savetxt("psoVymenaBatRokDom93.csv", batSOCProgress, delimiter=",")

In [11]:
np.savetxt("GridRokBezAbsDom114.csv", gridTotalProgress, delimiter=",")
np.savetxt("BatRokBezAbsDom114.csv", batSOCProgress, delimiter=",")

In [35]:
def sevas(j):
    j=3
    return 2, 3

j=4
c,d = sevas(j)
print(c,d)

2 3
